In [38]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression

In [139]:
df = pd.read_csv('../../datasets/processed_data/combined_features/IPCALAB.BO.csv')

In [140]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'low', 'high',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'inr=x_percent_change', 'cl=f_percent_change',
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change'],
      dtype='object', length=635)

In [141]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [142]:
df = create_custom_target(df)

In [143]:
df['yesterday_close']

1        58.779999
2        59.625000
3        60.000000
4        58.235001
5        57.294998
           ...    
3685    689.049988
3686    684.150024
3687    682.250000
3688    683.599976
3689    687.250000
Name: yesterday_close, Length: 3689, dtype: float64

In [144]:
df['ln_target'] 

1       0.020626
2       0.029335
3       0.021435
4       0.023169
5       0.003572
          ...   
3685    0.002030
3686    0.005175
3687    0.004752
3688    0.014450
3689    0.028333
Name: ln_target, Length: 3689, dtype: float64

In [145]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [146]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [147]:
df_without_date = df.drop(columns=["date"])

In [148]:
df_without_date.dropna(inplace=True)

In [149]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [150]:
window_size = 10

In [151]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# further split test set to have an hold out set to be used for backtesting
dev_data, test_data = train_test_split(test_data, train_size=0.5, shuffle=False)

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(dev_data, window_size, 'ln_target')  

In [152]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [153]:
rf_regressor = RandomForestRegressor(
    n_estimators=100,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
rf_regressor.fit(X_train_50, y_train)

RandomForestRegressor()

In [154]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling         window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [155]:
def evaluate_model(model, window, dev_data, dev_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(dev_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(dev_data, window, y_pred)

    dev_dates = dev_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': dev_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [160]:
predictions_df, mape, rmse = evaluate_model(rf_regressor, window_size, dev_data, dev_date, X_test_50, y_test)


In [161]:
predictions_df, mape, rmse

(           date       y_test       y_pred
 0    2020-06-23   837.500000   847.615074
 1    2020-06-24   836.750000   858.981946
 2    2020-06-25   842.500000   842.090234
 3    2020-06-26   854.575012   851.502935
 4    2020-06-29   853.799988   867.639473
 ..          ...          ...          ...
 354  2021-11-23  1055.699951  1067.117801
 355  2021-11-24  1050.000000  1083.315884
 356  2021-11-25  1068.750000  1078.155514
 357  2021-11-26  1042.500000  1044.695615
 358  2021-11-29  1071.099976  1043.582105
 
 [359 rows x 3 columns],
 0.0262494817881087,
 41.634644970550255)

In [158]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 2.62%


In [159]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

Repeat the process for the top 50 features without sentiment of IPCALAB.BO

In [126]:
df = pd.read_csv('../../datasets/processed_data/combined_features/IPCALAB.BO.csv')

In [127]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [128]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [129]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [130]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [131]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [132]:
df_without_date.dropna(inplace=True)

In [133]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# further split test set to have an hold out set to be used for backtesting
dev_data, test_data = train_test_split(test_data, train_size=0.5, shuffle=False)

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(dev_data, window_size, 'ln_target')  

In [134]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [135]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [164]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, dev_data, dev_date, X_test_50, y_test)

In [165]:
predictions_df, mape, rmse

(           date       y_test       y_pred
 0    2020-06-23   837.500000   848.022767
 1    2020-06-24   836.750000   853.722226
 2    2020-06-25   842.500000   841.937978
 3    2020-06-26   854.575012   850.796238
 4    2020-06-29   853.799988   865.689731
 ..          ...          ...          ...
 354  2021-11-23  1055.699951  1061.371332
 355  2021-11-24  1050.000000  1082.388447
 356  2021-11-25  1068.750000  1085.808016
 357  2021-11-26  1042.500000  1042.319710
 358  2021-11-29  1071.099976  1044.043471
 
 [359 rows x 3 columns],
 0.025479514004643312,
 42.168094381769606)

In [138]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 2.55%


In [107]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)